In [1]:
# 0) Instalar todas as dependências necessárias
%pip install langchain langchain-community openai pandas pandasql python-dotenv langchain_openai langchain-core


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Pichau\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
# 1) Importação das bibliotecas necessárias
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import StructuredTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os
import zipfile
import pandas as pd
from pandasql import sqldf
from io import TextIOWrapper
import re

# Variável global para armazenar o dataframe
df_global = None

#2) Criação da função das ferramentas
def carregar_arquivos(arquivo_zip_path: str):
    """
    **PRIMEIRA ETAPA SEMPRE**
    Quando necessário extrai os arquivos do formato .zip (Extrair,Extract,Descompactar,Unzip,Abrir,Open,Deszipar,Decompress,Desempacotar,Unarchive,Desarquivar)
    Exemplo: 'C:\\Users\Pichau\Downloads\\teste\\202401_NFs.zip'
    Exemplo2: 'C:\\Users\\ABC\\Documents\\arquivo.zip'
    Exemplo3: 'C:\\_______NOME.zip'
    """
    arquivo_zip_path = arquivo_zip_path.strip("'\"")
    global df_global # Declara que você vai modificar a variável global
    grupo_csv = {}
    
    with zipfile.ZipFile(arquivo_zip_path, 'r') as zip_ref:
        for arquivos in zip_ref.namelist():
            if arquivos.endswith('.csv'):
                with zip_ref.open(arquivos) as arq:
                    # Adicionado errors='ignore' para tentar lidar com problemas de codificação
                    df = pd.read_csv(TextIOWrapper(arq,'utf-8', errors='ignore'))
                    grupo_csv[arquivos] = df
            else:
                print(f"Sem suporte para o arquivo: '{arquivos}' não é um arquivo .csv")


        sorted_keys = sorted(grupo_csv.keys())
        primeiro_df = grupo_csv[sorted_keys[0]]
        segundo_df = grupo_csv[sorted_keys[1]]
       
        df_global = pd.merge(primeiro_df, segundo_df, on='CHAVE DE ACESSO', how='left')
        df_global.columns = df_global.columns.str.replace('_x$', '', regex=True)
        df_global = df_global.drop(columns=['MODELO_y','SÉRIE_y','NÚMERO_y','NATUREZA DA OPERAÇÃO_y','DATA EMISSÃO_y','CPF/CNPJ Emitente_y','RAZÃO SOCIAL EMITENTE_y','INSCRIÇÃO ESTADUAL EMITENTE_y','UF EMITENTE_y','MUNICÍPIO EMITENTE_y','CNPJ DESTINATÁRIO_y','NOME DESTINATÁRIO_y','UF DESTINATÁRIO_y','INDICADOR IE DESTINATÁRIO_y','DESTINO DA OPERAÇÃO_y','CONSUMIDOR FINAL_y','PRESENÇA DO COMPRADOR_y'])


        return f"Arquivos carregados e merged com sucesso! DataFrame pronto para análise com {len(df_global)} linhas e {len(df_global.columns)} colunas. As colunas disponíveis são: {list(df_global.columns)}."

def consulta_sql(query: str) -> str:
    """
    **SEGUNDA ETAPA SEMPRE**
    Executa uma consulta SQL no DataFrame df_global.
    """
    global df_global
    if df_global is None:
        return "O DataFrame global ainda não foi carregado."

    try:
        result = sqldf(query, {'df_global': df_global})
        return result.to_string(index=False)
    except Exception as e:
        return f"Erro na consulta: {e}"
    

def mapear_colunas(args: dict) -> str:
    """
    **TERCEIRA ETAPA OPCIONAL**
    Caso tenha dificuldades em encontrar o nome da coluna adequadamente, utilize essa ferramenta que mapeia os nomes das colunas do DataFrame global (df_global) para nomes mais simples e padronizados.
    Essa ferramenta deve ser usada quando a consulta SQL falhar por nomes de colunas incorretos ou complexos.
    """
    import re
    
    global df_global
    if df_global is None:
        return "O DataFrame global (df_global) ainda não foi carregado."

    colunas_originais = df_global.columns.tolist()
    
    # Cria dicionário de mapeamento simples
    mapeamento = {}
    for col in colunas_originais:
        col_nova = col.upper()
        col_nova = re.sub(r"[^A-Z0-9]", "_", col_nova)  # Substitui espaços/símbolos por _
        col_nova = re.sub(r"_+", "_", col_nova).strip("_")  # Remove múltiplos _
        mapeamento[col] = col_nova

    # Aplica o mapeamento
    df_global.columns = [mapeamento[col] for col in df_global.columns]
    
    retorno = "\n".join([f"{orig} -> {novo}" for orig, novo in mapeamento.items()])
    return f"Colunas renomeadas com sucesso:\n{retorno}"    



# 3) Criando as ferramentas usando StructuredTool
zip_tool = StructuredTool.from_function(
    func=carregar_arquivos,
    name="carregar_arquivos",
    description=carregar_arquivos.__doc__.strip()
)

query_tool = StructuredTool.from_function(
    func=consulta_sql,
    name="consulta_sql",
    description=consulta_sql.__doc__.strip()
)

map_tool = StructuredTool.from_function(
    func=mapear_colunas,
    name='mapear_colunas',
    description=mapear_colunas.__doc__.strip()
)


In [8]:
load_dotenv() # Carrega as variáveis de ambiente do arquivo .env

# 4) Escolhe o modelo selecionado e configuração de temperatura
llm = ChatOpenAI(
    model="openai/gpt-4o-mini", # Ou "gpt-3.5-turbo"
    base_url="https://openrouter.ai/api/v1",
    temperature=0.25,
    api_key = os.getenv("api_key"), # Certifique-se de que OPENAI_API_KEY está no seu .env
    max_tokens=500
)

# 5) Inicializando o agente com as ferramentas e o novo AgentType.OPENAI_FUNCTIONS (implicitamente usado por create_openai_tools_agent)
tools = [zip_tool, query_tool, map_tool]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente útil que pode extrair dados de arquivos ZIP, carregá-los em um DataFrame e responder a perguntas usando consultas SQL no DataFrame carregado. Sempre tente carregar o arquivo antes de tentar consultá-lo. Sempre que o usuário pedir para extrair um arquivo, verifique se o caminho completo e o nome do arquivo CSV são fornecidos. Se a consulta SQL falhar, considere usar a ferramenta 'mapear_colunas' antes de tentar a consulta SQL novamente."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)


In [9]:
# 6) Recebe a pergunta do usuário e executa o agente
# Inicializa o histórico de chat
chat_history = []

# Exemplo de uso 1: Extrair e encontrar o menor valor
user_input_1 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual o menor valor da coluna 'VALOR TOTAL'?"
print(f"Usuário: {user_input_1}")
response_1 = agent_executor.invoke({"input": user_input_1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_1), AIMessage(content=response_1["output"])])
print(f"Resposta: {response_1['output']}\n")


Usuário: Extraia o arquivo localizado em: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual o menor valor da coluna 'VALOR TOTAL'?


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QU

In [9]:

# Exemplo de uso 2: Extrair e encontrar o maior valor
user_input_2 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual o maior valor da coluna 'VALOR TOTAL'?"
print(f"Usuário: {user_input_2}")
response_2 = agent_executor.invoke({"input": user_input_2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_2), AIMessage(content=response_2["output"])])
print(f"Resposta: {response_2['output']}\n")


Usuário: Extraia o arquivo localizado em: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual o maior valor da coluna 'VALOR TOTAL'?


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QU

In [10]:

# Exemplo de uso 3: Extrair e encontrar o segundo maior valor
user_input_3 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual a 'RAZÃO SOCIAL EMITENTE' que teve o segundo maior valor na coluna 'VALOR TOTAL'?"
print(f"Usuário: {user_input_3}")
response_3 = agent_executor.invoke({"input": user_input_3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_3), AIMessage(content=response_3["output"])])
print(f"Resposta: {response_3['output']}\n")


Usuário: Extraia o arquivo localizado em: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual a 'RAZÃO SOCIAL EMITENTE' que teve o segundo maior valor na coluna 'VALOR TOTAL'?


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/S

In [16]:

# Exemplo de uso 4: Extrair e encontrar o tipo de produto com maior quantidade
user_input_4 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual 'NCM/SH (TIPO DE PRODUTO)' teve a maior 'QUANTIDADE'?"
print(f"Usuário: {user_input_4}")
response_4 = agent_executor.invoke({"input": user_input_4, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_4), AIMessage(content=response_4["output"])])
print(f"Resposta: {response_4['output']}\n")


Usuário: Extraia o arquivo localizado em: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Qual 'NCM/SH (TIPO DE PRODUTO)' teve a maior 'QUANTIDADE'?


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO

In [13]:

# Exemplo de uso 5: Teste com mapeamento de colunas
user_input_5 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Tente me dizer o total de QUANTIDADE para cada TIPO DE PRODUTO. Se houver erro, tente mapear as colunas e depois tente novamente."
print(f"Usuário: {user_input_5}")
response_5 = agent_executor.invoke({"input": user_input_5, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_5), AIMessage(content=response_5["output"])])
print(f"Resposta: {response_5['output']}\n")


Usuário: Extraia o arquivo localizado em: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip' e o arquivo CSV é '202401_NFs.csv'. Tente me dizer o total de QUANTIDADE para cada TIPO DE PRODUTO. Se houver erro, tente mapear as colunas e depois tente novamente.


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'D

In [10]:
user_input_6 = "Dado o arquivo, do que se tratam esse CSV?"
print(f"Usuário: {user_input_6}")
response_6 = agent_executor.invoke({"input": user_input_6, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_6), AIMessage(content=response_6["output"])])
print(f"Resposta: {response_6['output']}\n")

Usuário: Dado o arquivo, do que se tratam esse CSV?


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDADE', 'UNIDADE', 'VALOR UNITÁRIO', 'VALOR TOTAL'].O arquivo CSV contém informações sobre notas fiscais eletrônicas

In [21]:
user_input_7 = "Qual NOTA FISCAL teve maior valor, entendendo que Nota Fiscal é a Chave de Acesso'?"
print(f"Usuário: {user_input_7}")
response_7 = agent_executor.invoke({"input": user_input_7, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_7), AIMessage(content=response_7["output"])])
print(f"Resposta: {response_7['output']}\n")

Usuário: Qual NOTA FISCAL teve maior valor, entendendo que Nota Fiscal é a Chave de Acesso'?


> Entering new AgentExecutor chain...

Invoking: `carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`


Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDADE', 'UNIDADE', 'VALOR UNITÁRIO', 'VALOR TOTAL'].
Invoking: `consulta_sq